# 카카오 API 사용해서 주소 -> 경도, 위도, 우편번호

### 주소 검색하기
##### https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord

### 1. application key

In [98]:
# %pip install python-dotenv

from dotenv import load_dotenv
import os
load_dotenv()

# REST_API_KEY 환경변수에 저장했어요. 본인 키 사용
REST_API_KEY = os.environ.get("rest_app_key")

### 2. document : url, parems, headers

In [104]:
import pandas as pd
import requests

In [107]:
query = "경기도 성남시 분당구 불정로 90 KT 본사"

url = "https://dapi.kakao.com/v2/local/search/address.json"

params = {
    "query": query
}

headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
}

### 3. requests(url, params, headers) > response(json) : JSON(str)

In [108]:
response = requests.get(url, data=params, headers=headers)
response

<Response [200]>

In [109]:
response.text

'{"documents":[{"address":{"address_name":"경기 성남시 분당구 정자동 206","b_code":"4113510300","h_code":"4113557000","main_address_no":"206","mountain_yn":"N","region_1depth_name":"경기","region_2depth_name":"성남시 분당구","region_3depth_h_name":"정자3동","region_3depth_name":"정자동","sub_address_no":"","x":"127.114936252016","y":"37.3587823498654"},"address_name":"경기 성남시 분당구 불정로 90","address_type":"ROAD_ADDR","road_address":{"address_name":"경기 성남시 분당구 불정로 90","building_name":"KT본사","main_building_no":"90","region_1depth_name":"경기","region_2depth_name":"성남시 분당구","region_3depth_name":"정자동","road_name":"불정로","sub_building_no":"","underground_yn":"N","x":"127.114936252016","y":"37.3587823498654","zone_no":"13606"},"x":"127.114936252016","y":"37.3587823498654"}],"meta":{"is_end":true,"pageable_count":1,"total_count":1}}'

### 4. JSON(str) > prepocessing > data

In [110]:
data = response.json()["documents"][0]
data

{'address': {'address_name': '경기 성남시 분당구 정자동 206',
  'b_code': '4113510300',
  'h_code': '4113557000',
  'main_address_no': '206',
  'mountain_yn': 'N',
  'region_1depth_name': '경기',
  'region_2depth_name': '성남시 분당구',
  'region_3depth_h_name': '정자3동',
  'region_3depth_name': '정자동',
  'sub_address_no': '',
  'x': '127.114936252016',
  'y': '37.3587823498654'},
 'address_name': '경기 성남시 분당구 불정로 90',
 'address_type': 'ROAD_ADDR',
 'road_address': {'address_name': '경기 성남시 분당구 불정로 90',
  'building_name': 'KT본사',
  'main_building_no': '90',
  'region_1depth_name': '경기',
  'region_2depth_name': '성남시 분당구',
  'region_3depth_name': '정자동',
  'road_name': '불정로',
  'sub_building_no': '',
  'underground_yn': 'N',
  'x': '127.114936252016',
  'y': '37.3587823498654',
  'zone_no': '13606'},
 'x': '127.114936252016',
 'y': '37.3587823498654'}

In [111]:
x = data["x"]
y = data["y"]
zone_no = data["road_address"]["zone_no"]

print("경도: {}".format(x))
print("위도: {}".format(y))
print("우편번호: {}".format(zone_no))

경도: 127.114936252016
위도: 37.3587823498654
우편번호: 13606


# 카카오 API 사용해서 경도, 위도 -> 주소, 우편번호

### 좌표로 주소 변환하기

##### https://developers.kakao.com/docs/latest/ko/local/dev-guide#coord-to-address

### 1. document : url, parems, headers

In [112]:
import pandas as pd
import requests

In [113]:
# 앞에서 얻은 x, y 값 사용
url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={x}&y={y}"

headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
}

### 2. requests(url, params, headers) > response(json) : JSON(str)

In [114]:
response = requests.get(url, headers=headers)
response

<Response [200]>

In [115]:
response.text

'{"meta":{"total_count":1},"documents":[{"road_address":{"address_name":"경기도 성남시 분당구 불정로 90","region_1depth_name":"경기","region_2depth_name":"성남시 분당구","region_3depth_name":"","road_name":"불정로","underground_yn":"N","main_building_no":"90","sub_building_no":"","building_name":"KT본사","zone_no":"13606"},"address":{"address_name":"경기 성남시 분당구 정자동 206","region_1depth_name":"경기","region_2depth_name":"성남시 분당구","region_3depth_name":"정자동","mountain_yn":"N","main_address_no":"206","sub_address_no":"","zip_code":""}}]}'

### 3 JSON(str) > prepocessing > data

In [116]:
data = response.json()["documents"][0]
data

{'road_address': {'address_name': '경기도 성남시 분당구 불정로 90',
  'region_1depth_name': '경기',
  'region_2depth_name': '성남시 분당구',
  'region_3depth_name': '',
  'road_name': '불정로',
  'underground_yn': 'N',
  'main_building_no': '90',
  'sub_building_no': '',
  'building_name': 'KT본사',
  'zone_no': '13606'},
 'address': {'address_name': '경기 성남시 분당구 정자동 206',
  'region_1depth_name': '경기',
  'region_2depth_name': '성남시 분당구',
  'region_3depth_name': '정자동',
  'mountain_yn': 'N',
  'main_address_no': '206',
  'sub_address_no': '',
  'zip_code': ''}}

In [120]:
road_address = data["road_address"]["address_name"]
address = data["address"]["address_name"]
zone_no = data["road_address"]["zone_no"]

print("도로명 주소: {}".format(road_address))
print("지번 주소: {}".format(address))
print("우편번호: {}".format(zone_no))

도로명 주소: 경기도 성남시 분당구 불정로 90
지번 주소: 경기 성남시 분당구 정자동 206
우편번호: 13606
